# The key file is MCTR_Cfeed_n6r3K_269 which is the Arepo snapshot on a voronoi mesh.

Rowan

In [13]:
from arepy.read_write import binary_read as rsnap
from arepy.read_write import binary_write as wsnap
from arepy.utility import snap_utility as snut
from arepy.utility import cgs_constants as cgs
import numpy as np
import math
from scipy.interpolate import griddata, NearestNDInterpolator
import matplotlib.colors as mc
import matplotlib.pyplot as plt
import yt
from matplotlib.colors import LogNorm
from numpy import uint32, uint64, float64, float32
from numpy import linspace, array, logspace, sin, cos, pi, arange

In [14]:
########### GENERAL STUFF  ##########

#internal arepo units in cgs
ulength = 3.0856e20 #cm 
umass = 1.991e33 #solar mass in g
uvel = 1.0e5

utime = ulength/uvel
udensity = umass/ulength/ulength/ulength
uenergy= umass*uvel*uvel
ucolumn = umass/ulength/ulength

uparsec=ulength/3.0856e18
parsec = 3.0856e18
uMyr=utime/(60.*60.*24.*365.25*1.e6)


#make the images bigger
fig_size = plt.rcParams["figure.figsize"]
fig_size[0]=10
fig_size[1]=10
plt.rcParams["figure.figsize"] = fig_size
uparsec

100.0

In [15]:
number = 269
filenum = str(number).zfill(3)
ikeepsinks = 1
iwrite_newfile = 1
iremovetracers = 1
rcut = 1.0   #in which units are this value
iusealltracers = 0

#base = "/fs/computo56/other0/jesus/scripts_rowan/"
base = "/home/jesus/Dropbox/Galvan-Madrid/"
imbase = base
sim = "MCTR_Cfeed_n6r3K"
f = base + 'MCTR_Cfeed_n6r3K_' + filenum

rsnap.io_flags['mc_tracer']=True
rsnap.io_flags['time_steps']=True
rsnap.io_flags['sgchem']=True
rsnap.io_flags['variable_metallicity']=False

data, header = rsnap.read_snapshot(f)
time = header['time']
nparts = header['num_particles']

Loading file /home/jesus/Dropbox/Galvan-Madrid/MCTR_Cfeed_n6r3K_269
Particles [4946202       0 6479638   10244       0    1543]
Masses [0.        0.        5.0702798 0.        0.        0.       ]
Gas particles 4946202
Tracer particles 6479638
Sink particles 1543
Star particles 10244
Time =  [2.30057156]
Precision: Double
ID type:  <class 'numpy.uint32'>
Read mass
Read density
About to read tracers
ID type:  <class 'numpy.uint32'>
ID type:  <class 'numpy.uint32'>
[406397898 396482918 396477650 417455408 397597760 397244220 397541038
 413767580 396692634 396482771]
About to read timesteps
[0.00070365 0.00070365 0.00070365 0.00070365 0.00070365 0.00070365
 0.00070365 0.00070365 0.00070365 0.00070365]
Read in tdust


# ¿What are star particle ? what is the difference between star and sinks particles? why particles and not cells?

In [25]:
ngas = nparts[0]
print ("gas particles {}".format(ngas))
ntracer = nparts[2]
print ("tracer particles {}".format(ntracer))
nsinks = nparts[5] #numbers of sinks
print ("sinks particles = {}".format(nsinks))
total_part = ngas + nsinks  #number of gas cells and ¿sinks cells?

gas particles 4946202
tracer particles 6479638
sinks particles = 1543


# Pruebas

In [32]:
rho = data['rho'][0:ngas] # density of gas particles
pos = data['pos'][0:ngas,:] #posicion gas
# gas mass 
gasmass = data['mass'][0:ngas] # mass of gas cells

#"gas chemistry"?
xH2, xHp, xCO = data['chem'][0:ngas].T
xHp = xHp
xH2 = xH2
xCO = xCO
rho_cgs = rho * udensity
xHe=0.1

#constants
mp = 1.6726231e-24 
kb = 1.3806485e-16
xHI = 1 - xHp -2*xH2
yn = rho_cgs/((1. + 4.0 * xHe) * mp)


numtrace = data['numtrace'][0:ngas] #trace particles in gas cells
print (numtrace, "trace particles in each gas cell")

itrace,=np.where(numtrace > 0)
print ("index gas cell with trace particles")
print (itrace)

tracerno = numtrace[itrace]

#define the center
xcent=1400.
ycent=1400.
zcent=1400.

x,y,z = pos.T
xx=x[itrace]-xcent
yy=y[itrace]-ycent
zz=z[itrace]-zcent

# define sinks positions
if nsinks > 0 : #siks in this snapshot
    print(nsinks, 'Sinks are present')
    pos_sink = data['pos'][total_part-nsinks:total_part]
    xs,ys,zs = pos_sink.T
    xxs=xs-xcent
    yys=ys-ycent
    zzs=zs-zcent
#    print(xxs)

##############
# com  # and this is? #######  ### complex!!!
##############
if iusealltracers > 0 : #define the complex "position" using the tracer particles
    COX = np.sum(tracerno *xx)/np.sum(tracerno)
    COY = np.sum(tracerno *yy)/np.sum(tracerno)
    COZ = np.sum(tracerno *zz)/np.sum(tracerno)
else :
    COX = 86.0
    COY = -7.5
    COZ = 2.75
print("COM",COX,COY,COZ)

[0 0 0 ... 0 0 0] trace particles in each gas cell
index gas cell with trace particles
[  36044   36045   36046 ... 4941564 4941567 4944983]
1543 Sinks are present
COM 86.0 -7.5 2.75


In [33]:
print (data["pos"].shape,"positions")
print (ntracer,"trace particles" )
print ("total gas mass = {:.2e} M_sun" .format(gasmass.sum())) #in solar masses?
#print (data["mass"].size, "total mass")
print ("mass in sinks = {:.2e} M_sun". format(data["mass"][ngas:ngas+nsinks].sum()))
## the mass are in M_sun?

(4957989, 3) positions
6479638 trace particles
total gas mass = 2.19e+09 M_sun
mass in sinks = 3.60e+08 M_sun


# The low resolution is only in the complex C. The model is the feedback dominated.
The model include autogravity and the supernova are tied to sinks particles.

- **first step:** For the firs 150 Myr the gas is left to evolve only by the galactic potential. No autogravitation. Target mass $1000 \mathrm{M_\odot}$.

- **second step:** The simulation is turning on refinement for two spiral arm passages (70 Myr) within a 3 kpc box thath co-rotates with the fas centered on a galactic radius of 8 kpc. In this high resolution region the fas has a target mass of initially $100 \mathrm{M_\odot}$ fot the first 60 Myr, but it is further lowered to $10 \mathrm{M_\odot}$ for the final 10 Myr.


- **third step:** refine individual clouds by injecting massless Monte Carlo tracer particles into refions of interest thich are advectes ptobabilistically with the fas flow. Evolve other 4 Myr. 

Complex **C** is chosen to be inside an arm, and D is an inter-arm reg ion. Refined mass 8e5 solar mass.

Trace particles: 

### tracer particles arround the complex A,
plt.hist2d(xx,yy,bins=400)  #tracer particles
plt.plot(COX,COY,marker='o',color="red") 
plt.colorbar()
plt.grid()
plt.xlabel('x [kpc]',fontsize=18)
plt.ylabel('y [kpc]',fontsize=18)
plt.show()

""" xx, yy
plt.hist2d(xx,zz,bins=400)
plt.plot(COX,COY,marker='o',color="red")
plt.colorbar()
plt.grid()
plt.xlabel('x [100 pc]',fontsize=18)
plt.ylabel('z [100 pc]',fontsize=18)

plt.show()
"""

# extract the data in high resolution

In [34]:
### define cels cubic.
dx = np.abs(x-(COX+xcent))
dy = np.abs(y-(COY+ycent))
dz = np.abs(z-(COZ+zcent))

#condition to lower resolution cells
iextract, = np.where((dx<rcut)&(dy<rcut)&(dz<rcut)&(numtrace>0)&(gasmass>1.e-2))
x_ex = x[iextract]
y_ex = y[iextract]
z_ex = z[iextract]
rho_ex = rho[iextract] #not cgs
print ("Lower Box corner for extracted box region")
print ("min and max id", iextract.min(),iextract.max())

xminvalue = x_ex.min()
yminvalue = y_ex.min()
zminvalue = z_ex.min()

print ("min max positions x", x_ex.min(),x_ex.max())
print ("min max positions y", y_ex.min(),y_ex.max())
print ("min max poritions z", z_ex.min(),z_ex.max())

#trasladamos el centro, a 0,0,0
x_ex = x_ex - xminvalue
y_ex = y_ex - yminvalue
z_ex = z_ex - zminvalue

print (np.size(x_ex), "cells in the extracted region of size,", x_ex.max(),y_ex.max(),z_ex.max())
newbox = 2.*rcut

Lower Box corner for extracted box region
min and max id 89982 4935446
min max positions x 1485.0013317288488 1486.9976783216198
min max positions y 1391.5384985913367 1393.4999947645015
min max poritions z 1401.7616876917243 1403.7497082854516
265943 cells inthe extracted region of size, 1.996346592771033 1.9614961731647327 1.9880205937272422


In [37]:
#extract the sinks in extracted region
if nsinks > 0:
    dxs = np.abs(xs-(COX+xcent))
    dys = np.abs(ys-(COY+ycent))
    dzs = np.abs(zs-(COZ+zcent))
    iextract_sinks, = np.where((dxs < rcut)&(dys < rcut)&(dzs < rcut))

    if np.size(iextract_sinks) >0 :
    
        xs_ex = xs[iextract_sinks]
        ys_ex = ys[iextract_sinks]
        zs_ex = zs[iextract_sinks]
    
        xs_ex = xs_ex - xminvalue
        ys_ex = ys_ex - yminvalue
        zs_ex = zs_ex - zminvalue
    
        xs_ex[xs_ex<0] = 0.0
        ys_ex[ys_ex<0] = 0.0
        zs_ex[zs_ex<0] = 0.0
        print ("sinks mass ")
        print (data["mass"][iextract_sinks])
        print(np.size(xs_ex),'sinks in the extracted region of size,',newbox)
        print('Min max sink positions x', xs_ex.min(),xs_ex.max())
        print('Min max sink positions y', ys_ex.min(),ys_ex.max())
        print('Min max sink positions z', zs_ex.min(),zs_ex.max())
        
    else:
        
        print('zeroing sinks as none in the region')
        nsinks = 0

sinks mass 
[161.21905896 327.96123699 395.05313231 405.03600738]
4 sinks in the extracted region of size, 2.0
Min max sink positions x 0.5140918178760785 1.2092826715081628
Min max sink positions y 0.27969583065055303 1.2902407572125867
Min max sink positions z 0.3237788741212171 0.7839086931483052


#where are the sinks age?

## sinks data using arepy read sinks data:

In [46]:
#from arepy.read_write import binary_read as rsink
## read data sinks
time, SinkP = rsnap.read_sink_evolution_file(f)
SinkP

Loading file /home/jesus/Dropbox/Galvan-Madrid/MCTR_Cfeed_n6r3K_269
Time =  [3.0564786e-307]
Number of sink particles =  [0]


{'Pos': [],
 'Vel': [],
 'Acc': [],
 'Mass': [],
 'MassOld': [],
 'AccretionRadius': [],
 'FormationMass': [],
 'FormationTime': [],
 'ID': [],
 'HomeTask': [],
 'Index': [],
 'FormationOrder': [],
 'N_sne': [],
 'explosion_time': [],
 'MassStillToConvert': [],
 'AccretionTime': []}